In [1]:
import pandas as pd
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

In [2]:
train_law_par = pd.read_parquet(r"c:\Users\siren\Desktop\1507\dataset\hukuk_raw_dataset\train.parquet", engine="pyarrow")
wiki_law_par = pd.read_parquet(r"C:\Users\siren\Desktop\1507\dataset\ts_wikipedia\bpe_support.parquet")
concat_par = pd.concat([train_law_par, wiki_law_par]) 

In [3]:
concat_par = concat_par.iloc[:-1]
concat_par.drop(columns=["source"])
concat_par.reset_index(drop=True,inplace=True)

In [4]:
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space = False)

In [5]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('Ġis', (4, 7)),
 ('Ġan', (7, 10)),
 ('Ġexample', (10, 18)),
 ('!', (18, 19))]

In [6]:
trainer = trainers.BpeTrainer(vocab_size=50257, 
                              min_frequency = 1,
                              max_token_length = 7,
                              special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(concat_par["text"].astype(str), trainer=trainer)

In [7]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

In [8]:
tokenizer.model.save('.')
tokenizer.save("BPETokenizerWikiLaw.json")

In [10]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained(r'C:\Users\siren\Desktop\1507\GPT2_HF\train_tokenizer') #directory önemli merges, vocab ve tokenizer.config içinde olacak

In [11]:
text = "Example text to tokenize."
tokens = tokenizer.encode(text)
print(tokens)

[35, 85, 294, 13305, 355, 85, 81, 2395, 26426, 2865, 14]


In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [13]:
tokenizer.push_to_hub("sergeantson/1507_Law_Tokenizer")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\siren\anaconda3\envs\tubitak\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\siren\.cache\huggingface\hub\models--sergeantson--1507_Law_Tokenizer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/sergeantson/1507_Law_Tokenizer/commit/302d3f1edb8554d8f311ac9e1be5d41a38ea9e0b', commit_message='Upload tokenizer', commit_description='', oid='302d3f1edb8554d8f311ac9e1be5d41a38ea9e0b', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
sequence = "Adalet Mülkün Temelidir"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Adalet', 'ĠMÃ¼l', 'kÃ¼n', 'ĠTe', 'me', 'lidir']


In [15]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[15782, 2365, 810, 997, 212, 951]


In [16]:
decoded_string = tokenizer.decode(ids)
print(decoded_string)

Adalet Mülkün Temelidir


In [18]:
tokenizer.vocab_size

50257